In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10304,2024-02-22,Eua Nba,21:00,Philadelphia 76ers,New York Knicks,1.89,1.97,225.5,1.88,2.04,-1.5,2.01,2.15,M5LM74bl,0.529101,0.507614,0.531915,0.490196,0.036715,0.6,0.2,NaN,NaN,269.854,135.977181,0.503892,177.170,40.504332,0.228618,200.596,170.916,167.44,225.00,0.0,0.0,0.0,0.0,0.029310,0.057723,0.047594,-3.90,-0.780,-1.141026,0.637040,0.4,-0.237040,-0.62,-0.124,-7.822581,0.562990,0.5,-0.062990
10305,2024-02-22,Eua Nba,21:00,Toronto Raptors,Brooklyn Nets,1.78,2.11,230.5,1.85,2.09,-2.5,2.05,2.35,pMDD5wqn,0.561798,0.473934,0.540541,0.478469,0.035731,0.4,0.6,NaN,NaN,223.792,94.367575,0.421675,320.022,182.010949,0.568745,261.684,391.162,277.50,583.94,0.0,1.0,0.0,0.0,0.119972,0.086145,0.096424,-3.34,-0.668,-1.167665,0.504403,0.2,-0.304403,0.67,0.134,8.283582,0.383400,0.2,-0.183400
10306,2024-02-22,Eua Nba,21:30,Dallas Mavericks,Phoenix Suns,1.68,2.27,242.5,1.85,2.09,-2.5,1.97,2.55,vsGR6ODf,0.595238,0.440529,0.540541,0.478469,0.035767,0.6,0.6,NaN,NaN,190.300,64.104565,0.336861,176.524,32.383395,0.183450,207.734,210.332,131.08,207.20,0.0,0.0,0.0,0.0,0.211237,0.086145,0.181470,0.67,0.134,5.074627,0.602719,0.7,0.097281,0.09,0.018,70.555556,0.615882,0.6,-0.015882
10307,2024-02-22,Eua Nba,22:00,Chicago Bulls,Boston Celtics,3.67,1.31,225.5,1.88,2.04,7.5,1.97,1.07,IXEV5rT0,0.272480,0.763359,0.531915,0.490196,0.035838,0.8,0.2,NaN,NaN,313.890,109.307679,0.348236,149.054,7.811561,0.052408,223.904,170.024,328.95,151.04,1.0,0.0,0.0,0.0,0.670190,0.057723,0.418682,0.50,0.100,26.700000,0.593481,0.7,0.106519,2.02,0.404,0.767327,0.686281,0.8,0.113719
10308,2024-02-22,Eua Nba,22:00,Oklahoma City Thunder,Los Angeles Clippers,1.78,2.11,236.5,1.85,2.09,-2.5,2.05,2.35,AuXx4McD,0.561798,0.473934,0.540541,0.478469,0.035731,0.6,0.6,NaN,NaN,178.870,31.440428,0.175773,165.440,48.162739,0.291119,169.832,179.986,208.28,247.34,0.0,0.0,0.0,0.0,0.119972,0.086145,0.096424,0.54,0.108,7.222222,0.742066,0.9,0.157934,0.51,0.102,10.882353,0.631623,0.7,0.068377
10309,2024-02-22,Ásia Copa Asiática,09:00,Tailândia,Indonesia,1.72,2.02,155.5,1.85,1.85,-1.5,1.81,1.88,fmig7Xit,0.581395,0.495050,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.113440,0.000000,0.026828,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10310,2024-02-22,Eua Ncaa,00:00,New Mexico,Colorado State,1.30,3.70,157.5,1.91,1.91,-8.5,2.00,8.00,8K0l2Sup,0.769231,0.270270,0.523560,0.523560,0.039501,0.8,0.2,NaN,NaN,111.784,32.141079,0.287528,110.992,35.317196,0.318196,115.490,129.196,86.24,166.65,0.0,0.0,0.0,0.0,0.678823,0.000000,0.848528,-0.62,-0.124,-2.419355,0.812430,0.8,-0.012430,-3.63,-0.726,-3.719008,0.570830,0.5,-0.070830
10311,2024-02-22,Europa Eurobasket,16:30,Islândia,Hungria,1.76,2.00,157.5,1.81,1.93,-2.5,1.97,2.25,zs787bBs,0.568182,0.500000,0.552486,0.518135,0.068182,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.090269,0.045376,0.093834,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10312,2024-02-22,Eua Ncaa,21:00,Florida Gulf Coast,Kennesaw State,1.67,2.28,152.5,1.80,1.95,-4.5,2.05,3.15,tSDjUG4J,0.598802,0.438596,0.555556,0.512821,0.037399,0.4,0.6,NaN,NaN,157.994,86.460705,0.547240,156.278,78.684487,0.503490,115.512,154.986,75.60,292.60,0.0,0.0,0.0,0.0,0.218398,0.056569,0.299161,-0.24,-0.048,-13.958333,0.593125,0.5,-0.093125,-3.49,-0.698,-1.833811,0.525470,0.3,-0.225470
10313,2024-02-22

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Philadelphia 76ers,New York Knicks,225.5,1.88,1.0,Over
1,21:00,Eua Nba,Toronto Raptors,Brooklyn Nets,230.5,1.85,1.0,Over
2,21:30,Eua Nba,Dallas Mavericks,Phoenix Suns,242.5,1.85,1.0,Over
3,22:00,Eua Nba,Oklahoma City Thunder,Los Angeles Clippers,236.5,1.85,1.0,Over
4,00:00,Eua Ncaa,New Mexico,Colorado State,157.5,1.91,1.0,Over
5,16:30,Europa Eurobasket,Islândia,Hungria,157.5,1.81,1.0,Over
6,21:00,Eua Ncaa,Florida Gulf Coast,Kennesaw State,152.5,1.80,1.0,Over
7,21:00,Eua Ncaa,Northern Kentucky,Cleveland State,146.5,1.80,1.0,Over
8,22:00,Eua Ncaa,Abilene Christian,California Baptist,134.5,1.80,1.0,Over
9,22:00,Eua Ncaa,Georgia Southern,Texas State,136.5,1.80,1.0,Over
